# 6.3: Inspecting the `exp` directories

`run_train_phones.sh` will generate a new directory for each layer of the acoustic model in `exp`.  We will inspect their contents below.

In [1]:
ls exp

monophones          triphones          triphones_lda          triphones_sat
monophones_aligned  triphones_aligned  triphones_lda_aligned
